In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
data = pickle.load(open("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/tfidf_pca_result_final_500.pickle", "rb"))
X_pred = (pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_tfidf_pca_result_final_500.pickle")).drop(columns = ['doc_id'])

gc.collect()

25

In [4]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

# NN MODEL 4 is the best of the lot

## Model 4: Best Model

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import tensorflow as tf

x_train = tf.keras.utils.normalize(X_train, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)
print("Normalization Complete!")

model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten())   # input layer
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))   # output layer

model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
print("Model Ready")

model.fit(x_train, np.array(y_train), epochs=14, validation_data = (x_test, np.array(y_test)))

Normalization Complete!
Model Ready
Epoch 1/14

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

14593/14593 [==============================] - 47s 3ms/step - loss: 0.4955 - accuracy: 0.7716 - val_loss: 0.4690 - val_accuracy: 0.7835
Epoch 2/14
14593/14593 [==============================] - 46s 3ms/step - loss: 0.4747 - accuracy: 0.7814 - val_loss: 0.4764 - val_accuracy: 0.7755
Epoch 3/14
14593/14593 [==============================] - 46s 3ms/step - loss: 0.4660 - accuracy: 0.7857 - val_loss: 0.4642 - val_accuracy: 0.7885
Epoch 4/14
14593/14593 [=======

In [7]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []

pred = (model.predict(X_test))

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure(data = go.Scatter(x = cutoff, y = f1))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]

In [9]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_sent_500.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = (model.predict(X_pred))
probability_of_predictions = model.predict(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions >= max_cutoff])
result['probabilities'] = probability_of_predictions[predictions >= max_cutoff]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result.to_csv("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/result_tfidf_NER.csv")
result

,Sents,doc_id,probabilities
1,1869 of 1968. appeal under s. 116-a of the rep...,0.0,0.906612
2,"16 of 1967. s. v. gupte, m. m. tiwari, h. k. p...",0.0,0.394691
5,"rajasthan and k. b. mehta, for the state of ra...",0.0,0.675353
12,"she was declared elected on february 22, 1967....",0.0,0.347550
14,"838 article 191 of the constitution, which is ...",0.0,0.579311
...,...,...,...
7072,"21 'and 41 -of 1952, filed in the high court o...",49.0,0.457108
7074,they make large purchases of untanned hides an...,49.0,0.439515
7076,this is an unconstitutional delegation by the ...,49.0,0.519516
7079,hence the present appeals by the appellants un...,49.0,0.535094


# Other NN Models

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import tensorflow as tf

x_train = tf.keras.utils.normalize(X_train, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)
print("Normalization Complete!")

model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten())   # input layer
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 1
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 2
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 3
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))   # output layer

model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, np.array(y_train), epochs=5, validation_data = (x_test, np.array(y_test)))

Normalization Complete!
Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

14593/14593 [==============================] - 61s 4ms/step - loss: 0.4800 - accuracy: 0.7789 - val_loss: 0.4633 - val_accuracy: 0.7865
Epoch 2/5
14593/14593 [==============================] - 63s 4ms/step - loss: 0.4544 - accuracy: 0.7911 - val_loss: 0.4609 - val_accuracy: 0.7856
Epoch 3/5
14593/14593 [==============================] - 61s 4ms/step - loss: 0.4417 - accuracy: 0.7966 - val_loss: 0.4545 - val_accuracy: 0.7920
Epoch 4/5
14593/14593 [=======================

In [0]:
y_test.value_counts()

NameError: ignored

In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []

pred = (model.predict(X_test))

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure(data = go.Scatter(x = cutoff, y = f1))
fig.show()

max_cutoff = 0.305

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_sent_500.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = (model.predict(X_pred))
probability_of_predictions = model.predict(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions >= 0.325])
result['probabilities'] = probability_of_predictions[predictions >= 0.325]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

,Sents,doc_id,probabilities
1,1869 of 1968. appeal under s. 116-a of the rep...,0.0,0.879481
2,"16 of 1967. s. v. gupte, m. m. tiwari, h. k. p...",0.0,0.714431
5,"rajasthan and k. b. mehta, for the state of ra...",0.0,0.816623
12,"she was declared elected on february 22, 1967....",0.0,0.391657
13,"we are concerned only with one of them, namely...",0.0,0.399175
...,...,...,...
7054,"however, for the period, if any, served by him...",48.0,0.504205
7055,it will also not be necessary to remit his cas...,48.0,0.501379
7056,it is ordered and the appeals disposed of acco...,48.0,0.362696
7076,this is an unconstitutional delegation by the ...,49.0,0.631180


## Model 2

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten())   # input layer
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.2))
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))   # output layer

model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
print("Model Ready")

model.fit(x_train, np.array(y_train), epochs=10, validation_data = (x_test, np.array(y_test)))

Model Ready
Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

14593/14593 [==============================] - 65s 4ms/step - loss: 0.4869 - accuracy: 0.7758 - val_loss: 0.4693 - val_accuracy: 0.7837
Epoch 2/10
14593/14593 [==============================] - 66s 5ms/step - loss: 0.4639 - accuracy: 0.7860 - val_loss: 0.4623 - val_accuracy: 0.7915
Epoch 3/10
14593/14593 [==============================] - 65s 4ms/step - loss: 0.4534 - accuracy: 0.7918 - val_loss: 0.4565 - val_accuracy: 0.7912
Epoch 4/10
14593/14593 [==============================]

In [0]:
model.fit(x_train, np.array(y_train), epochs=10, validation_data = (x_test, np.array(y_test)))

Epoch 1/10
14593/14593 [==============================] - 66s 5ms/step - loss: 0.4120 - accuracy: 0.8091 - val_loss: 0.4621 - val_accuracy: 0.7968
Epoch 2/10
14593/14593 [==============================] - 65s 4ms/step - loss: 0.4087 - accuracy: 0.8106 - val_loss: 0.4690 - val_accuracy: 0.7978
Epoch 3/10
14593/14593 [==============================] - 65s 4ms/step - loss: 0.4048 - accuracy: 0.8126 - val_loss: 0.4729 - val_accuracy: 0.7975
Epoch 4/10
14593/14593 [==============================] - 66s 4ms/step - loss: 0.4019 - accuracy: 0.8139 - val_loss: 0.4662 - val_accuracy: 0.7980
Epoch 5/10
14593/14593 [==============================] - 65s 4ms/step - loss: 0.3987 - accuracy: 0.8156 - val_loss: 0.4690 - val_accuracy: 0.7969
Epoch 6/10
14593/14593 [==============================] - 66s 5ms/step - loss: 0.3957 - accuracy: 0.8166 - val_loss: 0.4760 - val_accuracy: 0.7975
Epoch 7/10
14593/14593 [==============================] - 65s 4ms/step - loss: 0.3933 - accuracy: 0.8175 - val_loss: 0

In [0]:
pred = model.predict(X_test)

In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []

pred = (model.predict(X_test))

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure(data = go.Scatter(x = cutoff, y = f1))
fig.show()

max_cutoff = 0.305

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_sent_500.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = (model.predict(X_pred))
probability_of_predictions = model.predict(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions >= 0.305])
result['probabilities'] = probability_of_predictions[predictions >= 0.305]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

,Sents,doc_id,probabilities
1,1869 of 1968. appeal under s. 116-a of the rep...,0.0,0.886553
2,"16 of 1967. s. v. gupte, m. m. tiwari, h. k. p...",0.0,0.522534
5,"rajasthan and k. b. mehta, for the state of ra...",0.0,0.746091
12,"she was declared elected on february 22, 1967....",0.0,0.505442
14,"838 article 191 of the constitution, which is ...",0.0,0.634930
...,...,...,...
7056,it is ordered and the appeals disposed of acco...,48.0,0.474442
7076,this is an unconstitutional delegation by the ...,49.0,0.658953
7079,hence the present appeals by the appellants un...,49.0,0.563060
7083,"this argument appears to us to be fallacious, ...",49.0,0.407895


## Model 3

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import tensorflow as tf

x_train = tf.keras.utils.normalize(X_train, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)
print("Normalization Complete!")

model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten())   # input layer
model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))   # output layer

model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, np.array(y_train), epochs=5, validation_data = (x_test, np.array(y_test)))

Normalization Complete!
Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

14593/14593 [==============================] - 87s 6ms/step - loss: 0.4996 - accuracy: 0.7707 - val_loss: 0.4779 - val_accuracy: 0.7808
Epoch 2/5
14593/14593 [==============================] - 87s 6ms/step - loss: 0.4784 - accuracy: 0.7802 - val_loss: 0.4705 - val_accuracy: 0.7848
Epoch 3/5
14593/14593 [==============================] - 86s 6ms/step - loss: 0.4709 - accuracy: 0.7844 - val_loss: 0.4671 - val_accuracy: 0.7880
Epoch 4/5
14593/14593 [=======================

In [0]:
model.fit(x_train, np.array(y_train), epochs=10, validation_data = (x_test, np.array(y_test)))

Epoch 1/10
14593/14593 [==============================] - 89s 6ms/step - loss: 0.4585 - accuracy: 0.7890 - val_loss: 0.4723 - val_accuracy: 0.7876
Epoch 2/10
14593/14593 [==============================] - 89s 6ms/step - loss: 0.4548 - accuracy: 0.7900 - val_loss: 0.4662 - val_accuracy: 0.7922
Epoch 3/10
14593/14593 [==============================] - 88s 6ms/step - loss: 0.4521 - accuracy: 0.7903 - val_loss: 0.4698 - val_accuracy: 0.7897
Epoch 4/10
14593/14593 [==============================] - 88s 6ms/step - loss: 0.4502 - accuracy: 0.7915 - val_loss: 0.4750 - val_accuracy: 0.7907
Epoch 5/10
14593/14593 [==============================] - 89s 6ms/step - loss: 0.4486 - accuracy: 0.7918 - val_loss: 0.4745 - val_accuracy: 0.7925
Epoch 6/10
14593/14593 [==============================] - 88s 6ms/step - loss: 0.4470 - accuracy: 0.7920 - val_loss: 0.4889 - val_accuracy: 0.7932
Epoch 7/10
14593/14593 [==============================] - 88s 6ms/step - loss: 0.4477 - accuracy: 0.7919 - val_loss: 0

KeyboardInterrupt: ignored

In [0]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []

pred = (model.predict(X_test))

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure(data = go.Scatter(x = cutoff, y = f1))
fig.show()

max_cutoff = 0.305